In [1]:
from nltk.tokenize import word_tokenize
from nltk.stem.snowball import EnglishStemmer

from flair.models import TextClassifier
from flair.data import Sentence

from newsapi import NewsApiClient
from datetime import datetime, timedelta, timezone

In [16]:
stemmer = EnglishStemmer()
api = NewsApiClient(api_key='54ad7b25e4444b559b023ecf81ed0ed2')
sia = TextClassifier.load('en-sentiment')

raw_data = api.get_everything(q='xrp')
raw_news_json = raw_data['articles']

def smart_tokenize_and_preprocess(text):
    words = word_tokenize(text)
    result = [stemmer.stem(token.lower()) for token in words]
    return result

def flair_prediction(x):
    # x = smart_tokenize_and_preprocess(x)
    sentence = Sentence(x)
    sia.predict(sentence)
    score = sentence.labels[0]
    if "POSITIVE" in str(score):
        return "POSITIVE"
    elif "NEGATIVE" in str(score):
        return "NEGATIVE"
    else:
        return "NEUTRAL"


In [19]:
current_datetime = datetime.now(timezone.utc)
pos = 0
neg = 0
neu = 0

for i in range(len(raw_news_json)):
    datetime_object = datetime.fromisoformat(raw_news_json[i]['publishedAt'].replace('Z', '+00:00'))
    
    if datetime_object > current_datetime - timedelta(days=2):
        description = raw_news_json[i]['description']
        content = raw_news_json[i]['content']
        
        if description is not None and content is not None:
            verdict = flair_prediction(description + content)
            if verdict == "POSITIVE": pos += 1
            elif verdict == "NEGATIVE": neg += 1
            else: neu += 1

            print(datetime_object.strftime("%A, %d %B %Y at %I:%M %p"))
            print(verdict)
            
print("RESULTS")
print(f"Positive Results: {pos}")
print(f"Negative Results: {neg}")
print(f"Neutral Results: {neu}")


Saturday, 02 December 2023 at 11:12 AM
NEGATIVE
Saturday, 02 December 2023 at 12:45 PM
POSITIVE
Saturday, 02 December 2023 at 05:54 PM
POSITIVE
RESULTS
Positive Results: 2
Negative Results: 1
Neutral Results: 0
